In [1]:
import partitura as pt
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import os
import pandas as pd

# Pitch Spelling with Partitura

Have you always been bad at spelling bee, do you find that spelling notes makes this even worse. Your time of struggling is over.... Today we going to teach a Model to learn how to *pitch* spell.

### Definition

Spelling a pitch relates to the system of naming notes by letters (A-G) and sharp(#) and flat (♭) signs - and sometimes double sharp and flat signs, resulting in names or 'spellings' like 'A♭', 'D#', 'F♭♭'.

Translating between frequencies in Hz and such names is non-trivial. You need to consider :

-  The 'concert pitch' you are taking as a reference
- The temperament in which the piece is played
- The overall key that the music would be notated in
- Use of the correct enharmonic equivalents for accidentals (Using the correct enharmonic equivalent, Purpose of double-sharps and double-flats?)

If translating between, say, MIDI note numbers and 'spelled' names, the first two steps can be skipped.

Spelled pitch names often have an octave number appended for disambiguation - e.g. 'A♭3', 'D#5'.


### Some Spelling algorithms

Partitura contains an implementation for a standard algorithm for Pitch Spelling. The algorithm in question is called ps13 created by Meredith and al.:

    The ps13 pitch spelling algorithm, D Meredith - Journal of New Music Research, 2006

Some notable algorithms and currect SOTA is PKSpell.

    PKSpell: Data-driven pitch spelling and key signature estimation
    F Foscarin, N Audebert, R Fournier-S'Niehotta, 2021


Let's first download a pitch spelling dataset.

In [2]:
!wget https://github.com/CPJKU/asap-dataset/archive/refs/heads/note_alignments.zip


--2022-10-27 12:26:52--  https://github.com/CPJKU/asap-dataset/archive/refs/heads/note_alignments.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/CPJKU/asap-dataset/zip/refs/heads/note_alignments [following]
--2022-10-27 12:26:52--  https://codeload.github.com/CPJKU/asap-dataset/zip/refs/heads/note_alignments
Resolving codeload.github.com (codeload.github.com)... 140.82.121.10
Connecting to codeload.github.com (codeload.github.com)|140.82.121.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘note_alignments.zip’

note_alignments.zip     [         <=>        ] 248.45M   782KB/s    in 4m 4s   

2022-10-27 12:30:57 (1.02 MB/s) - ‘note_alignments.zip’ saved [260516656]

unzip:  cannot find or open /content/note_alignments.zip, /content/note_alignments.zip.zip o

In [3]:
!unzip ./note_alignments.zip

Archive:  ./note_alignments.zip
32bd86d2d436b94e1a16643a31d397bd3a6b51d2
   creating: asap-dataset-note_alignments/
  inflating: asap-dataset-note_alignments/.gitignore  
   creating: asap-dataset-note_alignments/Bach/
   creating: asap-dataset-note_alignments/Bach/Fugue/
   creating: asap-dataset-note_alignments/Bach/Fugue/bwv_846/
  inflating: asap-dataset-note_alignments/Bach/Fugue/bwv_846/Shi05M.mid  
  inflating: asap-dataset-note_alignments/Bach/Fugue/bwv_846/Shi05M_annotations.txt  
   creating: asap-dataset-note_alignments/Bach/Fugue/bwv_846/Shi05M_note_alignments/
  inflating: asap-dataset-note_alignments/Bach/Fugue/bwv_846/Shi05M_note_alignments/note_alignment.tsv  
   creating: asap-dataset-note_alignments/Bach/Fugue/bwv_846/Shi05M_note_alignments/parangonada_data/
  inflating: asap-dataset-note_alignments/Bach/Fugue/bwv_846/Shi05M_note_alignments/parangonada_data/align.csv  
  inflating: asap-dataset-note_alignments/Bach/Fugue/bwv_846/Shi05M_note_alignments/parangonada_data

In [11]:
files = [(os.path.join(root, file), os.path.join(os.path.dirname(root), os.path.basename(root).split("_")[0]+".mid"),os.path.join(os.path.dirname(root), "xml_score.musicxml"), os.path.join(root, os.path.splitext(file)[0]+".match")) for root, dirs, files in os.walk("asap-dataset-note_alignments") for file in files if file.endswith("note_alignment.tsv")]

In [13]:
def produce_match(alignment_fn, mfn, sfn, match_name):
	"""
	Produce and Save Match.

	Parameters
	----------
	mfn : str
		Performance Midi File Path
	sfn : str
		Score musicxml File Path
	alignment_fn : str
		Alignment ".txt" file path
	match_name : str
		Path and Save Name.
	"""
	data = pd.read_csv(alignment_fn, sep="\t")

	alignment = list()
	for x in data[["xml_id", "midi_id"]].to_numpy():
		if x[1] == "deletion":
			dd = dict(label="deletion", score_id=x[0])
		# TODO for asap alignments to contain "n"
		elif x[0] == "insertion":
			dd = dict(label="insertion", performance_id=str(x[1]))
		else:
			dd = dict(label="match", score_id=x[0], performance_id=str(x[1]))
		alignment.append(dd)
	ppart = pt.load_performance_midi(mfn)
	# This may cause re-indexing.
	spart = pt.score.merge_parts(pt.load_musicxml(sfn))
	spart = pt.score.unfold_part_maximal(spart, ignore_leaps=False)
	pt.save_match(alignment, ppart, spart, match_name)

In [17]:
for (afn, mfn, sfn, match_name) in files:
    produce_match(afn, mfn, sfn, match_name)

/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/io/importmidi.py:140: UserWarning: change of Tempo to mpq = 500000  and resulting seconds per tick = 0.0013020833333333333at time: 0.0
  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/directions.py:533: UserWarning: error parsing "(   )" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:235: UserWarning: reference not found in
                            o_map: -- Tuplet start=n625 end=n630 start=None end=None, substituting None

  warnings.warn(
/home/manos/miniconda3/envs/partitura_tutorial/lib/python3.10/site-packages/partitura/utils/generic.py:235: UserWarning: reference not found in
                            o_map: -- Tuplet start=n922 end=n929 start=None end=None, substituting None

  warnin

TypeError: '<=' not supported between instances of 'int' and 'NoneType'

In [45]:
def tokenize_pitch_spelling(ps_note):
	step = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6}[ps_note["step"].item()]
	return step, ps_note["alter"], ps_note["octave"]

def create_data():
	_, _, _, match_files = zip(*files)
	data = list()
	labels = list()
	for match_file in match_files:
		performance, alignment, score = pt.load_match(match_file, create_score=True)
		matched_notes = [alignment[idx] for idx, d in enumerate(alignment) if d["label"] == "match"]
		pna = performance.note_array()
		sna = score.note_array(include_pitch_spelling=True)
		X, y = np.zeros((len(matched_notes), 3), dtype=float), np.zeros((len(matched_notes), 3), dtype=int)
		for idx, match_note in enumerate(matched_notes):
			X[idx] = np.lib.recfunctions.structured_to_unstructured(pna[np.where(pna["id"] == match_note["performance_id"])][["onset_sec", "duration_sec", "pitch"]])
			y[idx] = tokenize_pitch_spelling(sna[np.where(sna["id"] == match_note["score_id"])][["step", "alter", "octave"]])
		data.append(X)
		labels.append(y)
	return data, labels

/tmp/ipykernel_451760/4184092116.py:10: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  y[idx] = tokenize_pitch_spelling(sna[np.where(sna["id"] == match_note["score_id"])][["step", "alter", "octave"]])
/tmp/ipykernel_451760/4184092116.py:10: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  y[idx] = tokenize_pitch_spelling(sna[np.where(sna["id"] == match_note["score_id"])][["step", "alter", "octave"]])
/tmp/ipykernel_451760/4184092116.py:10: DeprecationWarning: setting an array element with a sequence. This was

array([[2, 1, 5],
       [3, 1, 5],
       [4, 1, 5],
       ...,
       [5, 1, 2],
       [0, 1, 4],
       [5, 1, 5]])

# Voice Separation